In [2]:
import time
import tensorflow as tf

In [3]:
#我们模拟一些原始输入数据
#（考虑从文件系统获取一些数据）
#每批次128个样本，每个样本包含1024个数据点
x_input_data = tf.random_normal([128, 1024], mean=0, stddev=1)


In [4]:
#我们建立我们的小模型：一个基本的两层神经网络与ReLU
with tf.variable_scope('queue'):
    q = tf.FIFOQueue(capacity=5, dtypes=tf.float32) 
    #我们使用“入队”操作，所以队列的1个元素是完整的批处理
    enqueue_op = q.enqueue(x_input_data)
    numberOfTheards = 1
    qr = tf.train.QueueRunner(q, [enqueue_op] * numberOfTheards)
    tf.train.add_queue_runner(qr)
    input = q.dequeue() #它取代了我们的输入占位符
    #我们也可以直接在图中计算y_true
    y_true = tf.cast(tf.reduce_mean(input, axis=1, keep_dims=True) > 0, tf.int32)
    

In [5]:
with tf.variable_scope('FullyConnected'):
    w = tf.get_variable('w',shape=[1024, 1024], initializer=tf.random_normal_initializer(stddev=1e-1))
    b = tf.get_variable('b',shape=[1024], initializer=tf.constant_initializer(0.1))
    z = tf.matmul(input, w) + b
    y = tf.nn.relu(z)
    
    w2 = tf.get_variable('w2',shape=[1024, 1], initializer=tf.random_normal_initializer(stddev=1e-1))
    b2 = tf.get_variable('b2',shape=[1], initializer=tf.constant_initializer(0.1))
    z = tf.matmul(y, w2) + b2

In [6]:
with tf.variable_scope('Loss'):
    losses = tf.nn.sigmoid_cross_entropy_with_logits(None, tf.cast(y_true, tf.float32), z)
    loss_op = tf.reduce_mean(losses)

In [7]:
with tf.variable_scope('Accruacy'):
    y_pred = tf.cast(z >0, tf.int32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(y_pred, y_true), tf.float32))
    accuracy = tf.Print(accuracy, data=[accuracy], message = 'accuracy:')
    

In [8]:
adam = tf.train.AdamOptimizer(1e-2)
train_op = adam.minimize(loss_op, name='train_op')

In [9]:
startTime = time.time()
with tf.Session() as sess:
    #初始化变量
    sess.run(tf.global_variables_initializer())
    
    #添加协调器
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    #检查训练前的准确性（不用feed_dict！）
    sess.run(accuracy)
    
    #训练
    for i in range(5000):
        #没有从Python采样，没有feed_dict！
        _, loss = sess.run([train_op, loss_op])
        
        #我们定期检查损失
        if i % 500 == 0:
            print('iter:%d --loss:%f'%(i, loss))
    
    
    #最后，我们检查我们的最终准确性
    sess.run(accuracy)
    
    coord.request_stop()
    coord.join(threads)

print('Time taken : %f'%(time.time() - startTime))
    

iter:0 --loss:2.666943
iter:500 --loss:0.521086
iter:1000 --loss:0.596342
iter:1500 --loss:0.401177
iter:2000 --loss:1.146182
iter:2500 --loss:0.934583
iter:3000 --loss:1.000453
iter:3500 --loss:1.204146
iter:4000 --loss:0.524669
iter:4500 --loss:1.016587
Time taken : 174.667655
